# Puzzles

## "Easy as ABC"
Source: https://twitter.com/Five_Triangles/status/1015738379454046208 (link now broken for some reason)

Rules:
Letters A,B,C must each appear exactly once in every row and column.
Outside letters must be closest to that letter in the respective row or column, e.g., C must be highest in column 1.

<img src = "img/01.jpg">

### My solution

In [1]:
from copy import deepcopy

In [2]:
def checkrow(row, req):
    '''Make sure a given row (or column) isn't breaking its two given requirements.'''

    # The row may contain up to 1 A, 1 B, 1 C, and the rest can be spaces
    n = len(row)
    if row.count('A') > 1 or row.count('B') > 1 or row.count('C') > 1 or row.count(' ') > n-3:
        return False
    
    # If the row has any letters, check that the first one matches the requirement
    rowstr = ''.join(row)
    if len(rowstr.strip()) > 0 and req[0] and rowstr.strip()[0] != req[0]:
        return False
    
    # If the row is full, check that the last letter matches the requirement
    if len(rowstr) == len(row) and req[1] and rowstr.strip()[-1] != req[1]:
        return False
    
    return True

def cols(grid):
    '''Get all the columns of a grid (essentially, transpose the grid).'''
    return [[e[i] for e in grid] for i in range(len(grid[0]))]

def col(grid, k):
    '''Get a single column of a grid.'''
    return [e[k] for e in grid]

def checkgrid(grid, row_reqs, col_reqs):
    '''Check that no rules are violated in an entire grid.'''
    if not all(checkrow(g, r) for g,r in zip(grid, row_reqs)):
        return False
    if not all(checkrow(g, r) for g,r in zip(cols(grid), col_reqs)):
        return False
    return True

def printgrid(grid):
    '''Neatly print out a grid.'''
    print(('\n' + '-' * (4*len(grid[0]) - 3) + '\n')
          .join([' | '.join(g) for g in grid]))  

# In each spot in a grid, letters will be tested in this order
cycle = {'':' ', ' ':'A', 'A':'B', 'B':'C'}
    
def solve_ABC(row_reqs, col_reqs):
    '''For an ABC grid problem specified by certain row and 
    column requirements, find and display all solutions.'''

    solutions = []
    
    # Size of the problem
    n = len(row_reqs)

    # The initial grid (all blank)
    grid = [[''] * n for _ in range(n)]

    # Starting position in the grid (upper left)
    currx, curry = 0, 0

    # Keep working until all possibilites are exhausted
    # (so if there are multiple solutions, we can find all of them)
    while curry >= 0:

        # At the current position in the grid, try the next letter
        try:
            grid[curry][currx] = cycle[grid[curry][currx]]

        # If there are no more letters to try, back up to the previous position
        except KeyError: 
            grid[curry][currx] = ''
            currx -= 1
            if currx < 0:
                currx  = n-1
                curry -= 1
            continue

        # If we have found a solution, save it
        if (currx, curry) == (n-1,n-1) and checkgrid(grid, row_reqs, col_reqs):
            solutions.append(deepcopy(grid))
            
        # Otherwise, if we're not currently breaking any rules, proceed to the next position
        elif (checkrow(grid[curry], row_reqs[curry]) and 
              checkrow(col(grid, currx), col_reqs[currx])):
            currx += 1
            if currx > n-1:
                currx  = 0
                curry += 1
    
    return solutions


In [3]:
soln = solve_ABC(
    row_reqs = [(None, None),
                ('A', None),
                (None, 'A'),
                (None, 'B'),
                (None, 'B'),
                (None, None)],
    col_reqs = [('C', None),
                ('C', 'A'),
                ('A', None),
                (None, None),
                (None, 'B'),
                (None, 'A')]
)

for s in soln: 
    printgrid(s)
    print('\n\n')


  | C |   |   | A | B
---------------------
  |   | A | B |   | C
---------------------
  | B |   |   | C | A
---------------------
C | A | B |   |   |  
---------------------
A |   |   | C | B |  
---------------------
B |   | C | A |   |  





# Puzzles

## "1 to 9" Puzzle #180713
Source: https://twitter.com/1to9puzzle/status/1018120012966621184

![The puzzle](https://pbs.twimg.com/media/DiEWj8gXkAIXFeE.jpg)

### My solution

In [4]:
from itertools import product, permutations

In [5]:
# Find the correct labels for all the sums we might possibly encounter
sum_dict = {}
max_sum = 9+8+7+6

# Evens and odds
for k in range(max_sum//2+1): sum_dict[2*k] = 'E'
for k in range(max_sum//2+1): sum_dict[2*k+1] = 'O'

# Primes (bleh!)
primes = list(range(2,max_sum+1))
for j in range(2,int(max_sum**.5)+1):
    for i in range(2,max_sum//j+1):
        try:
            primes.remove(i*j)
        except:
            pass
for k in primes: sum_dict[k] = 'P'
    
# Square, cube, triangular numbers
for k in range(int(max_sum**.5)+1): sum_dict[k**2] = 'S'
for k in range(int(max_sum**.3334)+1): sum_dict[k**3] = 'C'
for k in range(1,int((max_sum*2+1)**.5)+1): sum_dict[k*(k+1)//2] = 'T'
    

In [6]:
def checkgrid(grid):
    '''Determine whether a given grid obeys the rules.'''
    for i,j in product(range(7), range(7)):
        if (type(grid[i][j]) == str 
            and grid[i][j] != sum_dict[
                grid[i+1][j] + grid[i-1][j] 
                + grid[i][j+1] + grid[i][j-1]
            ]):
            return False
    return True

def printgrid(grid):
    '''Neatly print out a grid.'''
    print(('\n' + '-' * (4*len(grid[0]) - 3) + '\n')
          .join([' | '.join(str(x) for x in g) for g in grid]))  
    

In [7]:
# Set up the grid for the puzzle
grid = [[0] * 7 for _ in range(7)]

# Label the dark squares in the puzzle grid
grid[1][4] = 'T'
grid[2][1] = 'C'
grid[2][3] = 'T'
grid[3][2] = 'E'
grid[3][4] = 'E'
grid[4][3] = 'E'
grid[4][5] = 'E'
grid[5][2] = 'P'
grid[3][3] = 7

# Prepare to handle the light squares in the puzzle grid
lights = [(i,j) for i,j in product(range(1,6), range(1,6)) 
          if (i+j) % 2 == 0
          and (i,j) not in [(1,5),(5,1),(1,1),(5,5),(3,3)]]

# Try each possible way to fill the light squares, and show the solution(s)
for digits in permutations([1,2,3,4,5,6,8,9]):
    for (i,j),d in zip(lights, digits):
        grid[i][j] = d
    if checkgrid(grid):
        printgrid(grid)
        print('\n\n')
        

0 | 0 | 0 | 0 | 0 | 0 | 0
-------------------------
0 | 0 | 0 | 2 | T | 0 | 0
-------------------------
0 | C | 5 | T | 1 | 0 | 0
-------------------------
0 | 3 | E | 7 | E | 8 | 0
-------------------------
0 | 0 | 9 | E | 6 | E | 0
-------------------------
0 | 0 | P | 4 | 0 | 0 | 0
-------------------------
0 | 0 | 0 | 0 | 0 | 0 | 0





## "1 to 9" Puzzle #180907
Source: https://twitter.com/1to9puzzle/status/1037836364262395904

![The puzzle](https://pbs.twimg.com/media/DmciqVZW4AAruCj.jpg)

### My solution
I'll reuse some of the tools from previous problems:  
`sum_dict`, `cols()`, `printgrid()`, `itertools`

In [8]:
def checkgrid(grid, row_reqs, col_reqs, diag_reqs):
    '''Determine whether a given grid fits the requirements.'''
    
    # Check the rows and columns
    for i, row in enumerate(grid):
        if sum_dict[sum(row)] != row_reqs[i]:
            return False
    for i, col in enumerate(cols(grid)):
        if sum_dict[sum(col)] != col_reqs[i]:
            return False

    # Check the primary diagonal
    n = len(grid)
    if sum_dict[sum(grid[i][i] for i in range(n))] != diag_reqs[0]:
        return False

    # Check the secondary diagonal
    if sum_dict[sum(grid[i][n-i-1] for i in range(n))] != diag_reqs[1]:
        return False
    
    return True    

In [9]:
# Set up the grid for the puzzle
grid = [[0] * 3 for _ in range(3)]
grid[1][1] = 4

# Prepare to handle the blank squares in the puzzle grid
blanks = [(i,j) for i,j in product(range(3), range(3)) if i*j != 1]

# Try each possible way to fill the blanks, and show the solution(s)
for digits in permutations([1,2,3,5,6,7,8,9]):
    for (i,j),d in zip(blanks, digits):
        grid[i][j] = d
    if checkgrid(
        grid,
        row_reqs  = ('S','E','P'),
        col_reqs  = ('T','S','E'),
        diag_reqs = ('E','T')
    ):
        printgrid(grid)
        print('\n\n')
        

2 | 9 | 5
---------
7 | 4 | 1
---------
6 | 3 | 8



8 | 3 | 5
---------
1 | 4 | 7
---------
6 | 9 | 2





## "1 to 9" Puzzle #1836
Source: https://twitter.com/1to9puzzle/status/1038053770561826816

![The puzzle](https://pbs.twimg.com/media/DmfngHlXoAADa78.jpg)

### My solution

Again I'll reuse a few tools from earlier:   
`checkgrid()`, `printgrid()`, `itertools`  
In fact, it turns out that this problem requires almost nothing more than to redefine the `sum_dict`!

In [10]:
import inflect

In [11]:
# Find the correct labels for all the sums we might possibly encounter
# Each sum is represented by the first letter of the sum when 
# written out in English (e.g. represent 6 + 1 + 7 with "F")
sum_dict = {}
num2word = inflect.engine().number_to_words
for k in range(9+8+7+1): 
    sum_dict[k] = num2word(k)[0].upper()

In [12]:
# Set up the grid for the puzzle
grid = [[0] * 3 for _ in range(3)]
grid[1][1] = 2

# Prepare to handle the blank squares in the puzzle grid
blanks = [(i,j) for i,j in product(range(3), range(3)) if i*j != 1]

# Try each possible way to fill the blanks, and show any solution(s)
for digits in permutations([1,3,4,5,6,7,8,9]):
    for (i,j),d in zip(blanks, digits):
        grid[i][j] = d
    if checkgrid(
        grid,
        row_reqs  = ('N', 'E', 'F'),
        col_reqs  = ('S', 'S', 'T'),
        diag_reqs = ('S', 'S'),
    ):
        printgrid(grid)
        print('\n\n')
        

3 | 9 | 7
---------
5 | 2 | 4
---------
8 | 6 | 1



7 | 3 | 9
---------
4 | 2 | 5
---------
6 | 1 | 8





## "1 to 9" Puzzle #181013
Source: https://twitter.com/1to9puzzle/status/1050859935284649984

![The puzzle](https://pbs.twimg.com/media/DpVniaiWsAAwfzx.jpg)

### My solution

Again, this problem requires only a redefinition of the `sum_dict`.

In [13]:
import roman_numerals
def numeral(num):
    s = roman_numerals.convert_to_numeral(num)
    for a, b in roman_numerals.SHORTENINGS:        
        s = s.replace(b, a) # undo some fancy stuff we don't want
    return s

In [14]:
# Find the correct labels for all the sums we might possibly encounter
# Each sum is represented by the first letter of the sum when 
# written out in English (e.g. represent 6 + 1 + 7 with "F")
sum_dict = {}
for k in range(9+8+7+1): 
    sum_dict[k] = len(numeral(k))

In [16]:
# Set up the grid for the puzzle
grid = [[0] * 3 for _ in range(3)]
grid[1][1] = 4

# Prepare to handle the blank squares in the puzzle grid
blanks = [(i,j) for i,j in product(range(3), range(3)) if i*j != 1]

# Try each possible way to fill the blanks, and show any solution(s)
for digits in permutations([1,2,3,5,6,7,8,9]):
    for (i,j),d in zip(blanks, digits):
        grid[i][j] = d
    if checkgrid(
        grid,
        row_reqs  = (3, 3, 3),
        col_reqs  = (2, 2, 1),
        diag_reqs = (3, 4),
    ):
        printgrid(grid)
        print('\n\n')
        

2 | 9 | 1
---------
5 | 4 | 3
---------
8 | 7 | 6





## "Skyscrapers"
Source: https://twitter.com/solvemymaths/status/1038308216264896518

![The puzzle part 1](https://pbs.twimg.com/media/DmjP0LaXgAAPh0X.jpg)
![The puzzle part 2](https://pbs.twimg.com/media/DmjP0ahX0AEUN2K.jpg)


### My solution

We'll reuse some code from before verbatim, but let's pretend we're starting from scratch.

In [13]:
from copy import deepcopy

In [14]:
def count_maxes(l):
    '''Count the number of times a list of positive numbers reaches a new maximum.'''
    mx = l[0]
    cnt = 1
    for i in l[1:]:
        if i and i > mx:
            cnt += 1
            mx = i
    return cnt

def checkrow(row, req):
    '''Make sure a given row (or column) isn't breaking its two given requirements.'''

    # The row may contain up to one of each number
    n = len(row)
    if max(row.count(d+1) for d in range(n)) > 1:
        return False
    
    # If the row has any numbers, check that they do not exceed the first requirement
    if req[0] and any(row) and count_maxes(row) > req[0]:
        return False
    
    # If the row is full, check that both requirements are met exactly
    if req[0] and all(row) and count_maxes(row) != req[0]:
        return False
    if req[1] and all(row) and count_maxes(row[::-1]) != req[1]:
        return False
    
    return True

def cols(grid):
    '''Get all the columns of a grid (essentially, transpose the grid).'''
    return [[e[i] for e in grid] for i in range(len(grid[0]))]

def col(grid, k):
    '''Get a single column of a grid.'''
    return [e[k] for e in grid]

def checkgrid(grid, row_reqs, col_reqs):
    '''Check that no rules are violated in an entire grid.'''
    if not all(checkrow(g, r) for g,r in zip(grid, row_reqs)):
        return False
    if not all(checkrow(g, r) for g,r in zip(cols(grid), col_reqs)):
        return False
    return True

def printgrid(grid):
    '''Neatly print out a grid.'''
    print(('\n' + '-' * (4*len(grid[0]) - 3) + '\n')
          .join([' | '.join(str(i) for i in g) for g in grid]))  

    
def solve_skyscraper(row_reqs, col_reqs, fill_coords = [], fill_vals = []):
    '''For a skyscraper problem specified by certain row and 
    column requirements, find and display all solutions.'''

    # Size of the problem
    n = len(row_reqs)

    # In each spot in a grid, numbers will be tested in order
    cycle = {None : 1}
    for i in range(1,n): cycle[i] = i+1

    solutions = []

    # The initial grid with given values filled in
    grid = [[None] * n for _ in range(n)]
    for (i,j), v in zip(fill_coords, fill_vals):
        grid[i][j] = v
    
    # Starting position in the grid (upper left)
    currx, curry = 0, 0

    # Keep working until all possibilites are exhausted
    # (so if there are multiple solutions, we can find all of them)
    while curry >= 0:
                
        # At the current position in the grid, try the next letter
        try:
            grid[curry][currx] = cycle[grid[curry][currx]]

        # If there are no more letters to try, back up to the previous position
        # (and keep backing up if a filled spot is reached)
        except KeyError: 
            grid[curry][currx] = None
            retreat = True
            while retreat:
                currx -= 1
                if currx < 0:
                    currx  = n-1
                    curry -= 1
                retreat = ((curry, currx) in fill_coords)
            continue

        # If we have found a solution, save it
        if (currx, curry) == (n-1,n-1) and checkgrid(grid, row_reqs, col_reqs):
            solutions.append(deepcopy(grid))
            
        # Otherwise, if we're not currently breaking any rules, advance to the next position
        # (and keep advancing if a filled spot is reached)        
        elif (checkrow(grid[curry], row_reqs[curry]) and 
              checkrow(col(grid, currx), col_reqs[currx])):
            advance = True
            while advance:
                currx += 1
                if currx > n-1:
                    currx  = 0
                    curry += 1
                advance = ((curry, currx) in fill_coords)
    
    return solutions


#### First puzzle

In [15]:
soln = solve_skyscraper(
    row_reqs = [(2,3),(2,2),(2,1),(1,4)],
    col_reqs = [(4,1),(1,2),(2,2),(3,2)]
)

for s in soln: 
    printgrid(s)
    print('\n\n')


1 | 4 | 3 | 2
-------------
2 | 1 | 4 | 3
-------------
3 | 2 | 1 | 4
-------------
4 | 3 | 2 | 1





#### Second puzzle

In [16]:
soln = solve_skyscraper(
    row_reqs = [(3,None),(None,None),(2,None),(None,None)],
    col_reqs = [(3,None),(None,None),(2,None),(None,None)],
    fill_coords = [(1,2)], fill_vals = [1]
)

for s in soln: 
    printgrid(s)
    print('\n\n')


1 | 3 | 2 | 4
-------------
3 | 4 | 1 | 2
-------------
2 | 1 | 4 | 3
-------------
4 | 2 | 3 | 1





## Generating new "skyscrapers" puzzles
It would be interesting to me to see a skyscraper puzzle with several distinct solutions...

In [17]:
# For drawing random numbers
from random import randrange

In [18]:
def printreqs(row_reqs, col_reqs):
    '''Neatly print out an unsolved puzzle grid.'''
    n = len(row_reqs)
    print(' ' + '   '.join(str(r[0]) if r[0] else ' ' for r in col_reqs))
    for i in range(n):
        print(str(row_reqs[i][0]) if row_reqs[i][0] else ' ', end = '')
        print(' | '.join(' ' for j in range(n)), end = '')  
        print(str(row_reqs[i][1]) + '  ' if row_reqs[i][1] else '')            
        if i < n-1: print(' ' + '-' * (4*n - 3))
    print(' ' + '   '.join(str(r[1]) if r[1] else ' ' for r in col_reqs))


In [19]:
def rn():
    '''Returns None, 1, 2, 3, or 4.'''
    val = randrange(0, 5)
    return val if val else None

# Look for puzzles with at least 3 solutions
soln = []
while len(soln) < 3:
    row_reqs = [(rn(),rn()),(rn(),rn()),(rn(),rn()),(rn(),rn())]
    col_reqs = [(rn(),rn()),(rn(),rn()),(rn(),rn()),(rn(),rn())]
    soln = solve_skyscraper(row_reqs, col_reqs)

In [20]:
printreqs(row_reqs, col_reqs)

 3       2   2
2  |   |   |  
 -------------
   |   |   |  
 -------------
   |   |   |  1  
 -------------
   |   |   |  
 1       2   2


In [21]:
for s in soln: 
    printgrid(s)
    print('\n\n')

1 | 4 | 2 | 3
-------------
3 | 1 | 4 | 2
-------------
2 | 3 | 1 | 4
-------------
4 | 2 | 3 | 1



1 | 4 | 2 | 3
-------------
3 | 2 | 4 | 1
-------------
2 | 3 | 1 | 4
-------------
4 | 1 | 3 | 2



1 | 4 | 3 | 2
-------------
3 | 2 | 4 | 1
-------------
2 | 3 | 1 | 4
-------------
4 | 1 | 2 | 3



2 | 4 | 1 | 3
-------------
1 | 3 | 4 | 2
-------------
3 | 1 | 2 | 4
-------------
4 | 2 | 3 | 1



2 | 4 | 1 | 3
-------------
3 | 1 | 4 | 2
-------------
1 | 3 | 2 | 4
-------------
4 | 2 | 3 | 1



2 | 4 | 1 | 3
-------------
3 | 2 | 4 | 1
-------------
1 | 3 | 2 | 4
-------------
4 | 1 | 3 | 2





## "A to Z" Puzzle #180910

Source: https://twitter.com/1to9puzzle/status/1039140873773219841  
![The puzzle](https://pbs.twimg.com/media/DmvFCgZW0AA0uw5.jpg)

### My solution

No `import` statements or previous functions are needed. The solution is fairly straightforward.

In [22]:
def expr(grid, p):
    '''Convert a path through the grid into a mathematical expression.'''
    return ' '.join(grid[i][j] for i,j in p[1:-1])

def check_path(grid, p):
    '''Check whether a path produces a true equation.'''
    e = expr(grid, p)
    return '==' in e and eval(e)

def solve_AtoB(grid):
    '''Find all paths through the given grid, starting 
       at "A" and ending at "B" and forming a true equation.'''
    
    # A path through the grid 
    # (initially it should only have the location of "A")    
    path = [(i, row.index('A')) for i, row in enumerate(grid) if 'A' in row]   
    assert len(path) == 1
    
    # List of directions taken
    dirs = ['']

    # Solutions to be returned
    turns, eqns = [], []

    # Directions will be tried in the following order
    next_dir = {'':'R', 'R':'D', 'D':'L', 'L':'U'}

    # Keep trying paths until there are none left
    # (so all solutions should be found)
    while (grid[path[-1][0]][path[-1][1]], dirs[-1]) != ('A', 'U'):
        curr_y, curr_x = path[-1]

        # Try the next direction at the current position in the grid
        try:
            dirs[-1] = next_dir[dirs[-1]]

        # If all directions have been tried, back up
        except:
            path = path[:-1]
            dirs = dirs[:-1]
            continue

        # Move forward in the chosen direction
        if dirs[-1]   == 'R': curr_x += 1
        elif dirs[-1] == 'D': curr_y += 1
        elif dirs[-1] == 'L': curr_x -= 1
        elif dirs[-1] == 'U': curr_y -= 1

        # Add the next position to the path, if it's inside 
        # the grid and it hasn't already been visited
        if ((curr_y, curr_x) not in path
            and 0 <= curr_y < len(grid)
            and 0 <= curr_x < len(grid[0])):
            path.append((curr_y, curr_x))
            dirs.append('')            

            # If we reach the "B" square, check whether the path 
            # is valid and then back up to look for others
            if grid[curr_y][curr_x] == 'B':
                if check_path(grid, path): 
                    turns.append(''.join(dirs))
                    eqns.append(expr(grid, path))
                path = path[:-1]
                dirs = dirs[:-1]
                
    return eqns, turns

In [23]:
eqns, turns = solve_AtoB([
    ['A', '5', '+',  '1', '/'],
    ['4', '-', '2',  '*', '1'],
    ['/', '8', '==', '2', '+'],
    ['5', '+', '8',  '+', '2'],
    ['/', '2', '*',  '2', 'B']
])

# Report the size of the solutions set
print('There are %d valid paths through the grid, producing %d distinct equations.' 
     % (len(turns), len(set(eqns))))

# List the solutions in order of size
sort_index = sorted([(len(t), e, i) for i, (t, e) in enumerate(zip(turns,eqns))])
for _, _, i in sort_index:
    print('%s (%s)' % (eqns[i].replace('==','=').replace('*','×'), turns[i]))

There are 34 valid paths through the grid, producing 22 distinct equations.
4 - 8 + 8 = 2 + 2 (DRDDRURRDD)
4 - 8 + 8 = 2 + 2 (DRDDRURDRD)
4 - 8 + 8 = 2 + 2 (DRDDRURDDR)
4 - 8 + 8 = 2 × 1 + 2 (DRDDRURURDDD)
4 - 8 + 8 = 2 × 1 + 2 (DRDDRUURRDDD)
4 - 8 + 8 = 2 × 1 / 1 + 2 (DRDDRURUURDDDD)
4 - 8 + 8 = 2 × 1 / 1 + 2 (DRDDRUURURDDDD)
5 + 1 / 1 + 2 × 2 = 8 + 2 (RRRRDDLULDDRRD)
5 + 1 / 1 + 2 × 2 = 8 + 2 (RRRRDDLULDDRDR)
5 - 8 + 8 = 2 + 1 × 1 + 2 (RDDDRUUURDRDDD)
5 - 8 + 8 = 2 + 1 / 1 + 2 (RDDDRUUURRDDDD)
4 - 2 × 1 + 2 + 8 = 8 + 2 × 2 (DRRRRDDLLULDDRRR)
4 - 2 × 1 + 2 + 8 = 8 + 2 × 2 (DRRRRDLDLULDDRRR)
4 - 2 + 1 × 2 + 8 = 8 + 2 × 2 (DRRURDDDLULDDRRR)
4 - 2 × 1 / 1 + 2 + 8 = 8 + 2 × 2 (DRRRURDDDLLULDDRRR)
4 - 2 × 1 / 1 + 2 + 8 = 8 + 2 × 2 (DRRRURDDLDLULDDRRR)
4 - 2 + 1 / 1 × 2 + 8 = 8 + 2 × 2 (DRRURRDLDDLULDDRRR)
5 + 1 × 1 + 2 + 2 = 2 - 8 + 8 × 2 (RRRDRDDLULULDDRDRR)
5 + 1 / 1 + 2 + 2 × 2 = 8 + 2 × 2 (RRRRDDDLUULDDLDRRR)
5 + 1 / 1 + 2 + 2 × 2 = 8 + 2 × 2 (RRRRDDDLUULDLDDRRR)
5 + 1 / 1 + 2 + 2 = 2 

## Generating new "A to Z" puzzles
This is a really cool kind of puzzle; I have ideas for several variations to try to generate.

### Idea 1: Get away from the corners
My solution code should be able to handle puzzles where "A" and/or "B" are not in corners of the puzzle, and "=" is not in the middle. Let's create a puzzle like that, with solutions.

In [24]:
def printgrid(grid):
    '''Neatly print out a grid.'''
    print(('\n' + '-' * (4*len(grid[0]) - 3) + '\n')
          .join([' | '.join(i.replace('==','=').replace('*','×') for i in g) for g in grid]))  

    
    
pipe_dict = {
    'UU' : '│', 'UL' : '┐', 'UR' : '┌', 'DD' : '│',
    'DL' : '┘', 'DR' : '└', 'LU' : '└', 'LD' : '┌',
    'LL' : '─', 'RU' : '┘', 'RD' : '┐', 'RR' : '─',
    'R'  : 'B', 'D'  : 'B', 'L'  : 'B', 'U'  : 'B' 
}    
    
try:
    import pandas as pd
    from IPython.display import display, HTML
    pandas_display = True
except:
    pandas_display = False
    
def printpath(path, n = None, start = (0,0)):
    '''Display a path through a grid in (sort of) graphical form.'''
    
    # Set up the solution grid to be displayed
    if n is None: n = max(start) + 1
    grid = [[' '] * n for _ in range(n)]

    # Fill in the starting position
    i,j = start
    grid[i][j] = 'A'
    
    # Fill in the rest of the path
    for k in range(len(path)):
        if   path[k] == 'R': j += 1
        elif path[k] == 'L': j -= 1
        elif path[k] == 'U': i -= 1
        elif path[k] == 'D': i += 1
            
        # Expand the grid as necessary
        if i == n or j == n:
            grid.append([' ']*n)
            for row in grid: row.append(' ')
            n += 1
            
        # Put the correct symbol in the current space
        grid[i][j] = pipe_dict[path[k:k+2]]
        
    if pandas_display:
        display(HTML(pd.DataFrame(grid).to_html(header = False, index = False)))
    else:
        print('\n'.join(' '.join(row) for row in grid))
    
    
def build_grid(n, max_number,
               start = None, stop = None, eq = None,
               operators = '+-/*'):
    '''Create an n-by-n problem at random.
       e.g.: build_grid(5, 9, start = (0,0), stop = (4,4), eq = (2,2)) 
             would return a 5-by-5 grid with "A" and "B" in opposite 
             corners, "=" in the middle, with numbers no larger than 9.
       CAUTION: A solution is not guaranteed to exist!'''

    # The grid to be returned
    grid = [[0] * n for _ in range(n)]
    
    # Places to put numbers
    number_spots = [(i,j) for i,j 
                    in product(range(n), range(n))
                    if (i+j) % 2 == 1]
    
    # Places to put operators like +, -, /, *
    operator_spots = [(i,j) for i,j 
                      in product(range(n), range(n))
                      if (i+j) % 2 == 0]
    
    # Fill in random numbers
    for i,j in number_spots:
        grid[i][j] = str(randrange(1, max_number+1))

    # Fill in random operators
    for i,j in operator_spots:
        grid[i][j] = operators[randrange(0, len(operators))]
    
    # Put "A" somewhere
    if start:
        grid[start[0]][start[1]] = 'A'
    else:
        i,j = operator_spots[randrange(0, len(operator_spots))]
        grid[i][j] = 'A'

    # Put "B" somewhere
    if stop:
        grid[stop[0]][stop[1]] = 'B'
    else:
        while grid[i][j] == 'A':
            i,j = operator_spots[randrange(0, len(operator_spots))]
        grid[i][j] = 'B'
        
    # Put "=" somewhere
    if eq:
        grid[eq[0]][eq[1]] = '=='
    else:
        while grid[i][j] in ('A', 'B'):
            i,j = operator_spots[randrange(0, len(operator_spots))]
        grid[i][j] = '=='
    
    return grid


In [25]:
# Randomly generate puzzles until we get one with 
# a reasonable number of solutions, then show it
turns = []
while len(turns) < 1 or len(turns) > 10:
    grid = build_grid(5, 9)
    eqns, turns = solve_AtoB(grid)

printgrid(grid)
print('\n\n')

# Report the size of the solutions set
print('There are %d valid paths through the grid, producing %d distinct equations.' 
     % (len(turns), len(set(eqns))))

# List the solutions in order of size
sort_index = sorted([(len(t), e, i) for i, (t, e) in enumerate(zip(turns,eqns))])
for _, _, i in sort_index:
    print('%s (%s)' % (eqns[i].replace('==','=').replace('*','×'), turns[i]))
    
# Illustrate the longest solution
print('\n\n')
start = [(i, row.index('A')) for i, row in enumerate(grid) if 'A' in row][0]
printpath(turns[sort_index[-1][-1]], start = start)

× | 6 | / | 4 | -
-----------------
8 | - | 6 | A | 3
-----------------
B | 3 | × | 7 | -
-----------------
9 | - | 8 | = | 8
-----------------
- | 7 | - | 7 | /



There are 7 valid paths through the grid, producing 7 distinct equations.
7 - 8 = 8 - 9 (DRDLLLLU)
3 - 7 = 7 - 8 - 3 (RDLDDLULUL)
3 - 7 = 7 - 8 × 6 / 6 - 3 (RDLDDLUUUULDDL)
3 - 7 = 8 × 6 / 6 - 3 - 9 (RDLDLUUULDDDLU)
3 - 8 = 7 × 6 / 6 - 3 - 9 (RDDLULUULDDDLU)
6 - 8 × 6 / 4 - 3 - 8 = 7 - 8 × 3 (LLLURRRRDDDLDLUULL)
6 - 8 × 6 / 4 - 3 - 8 = 7 - 8 - 7 - 9 (LLLURRRRDDDLDLULDLUU)





┌,─,─,─,┐
└,─,─,A,│
B,,,,│
│,┌,┐,┌,┘
└,┘,└,┘,


### Idea 2: A single solution
If the puzzle has exactly one solution and no more, will it be especially difficult? Try it out!

In [26]:
# Randomly generate puzzles until we get one with 
# a single solution, then show it
turns = []
while len(turns) != 1:
    grid = build_grid(5, 9, (0,0), (4,4), (2,2))
    eqns, turns = solve_AtoB(grid)

printgrid(grid)
print('\n\n')

# Report the size of the solutions set
print('There are %d valid paths through the grid, producing %d distinct equations.' 
     % (len(turns), len(set(eqns))))

# List the solutions in order of size
sort_index = sorted([(len(t), e, i) for i, (t, e) in enumerate(zip(turns,eqns))])
for _, _, i in sort_index:
    print('%s (%s)' % (eqns[i].replace('==','=').replace('*','×'), turns[i]))
    
# Illustrate the only solution
print('\n\n')
printpath(turns[sort_index[-1][-1]])

A | 3 | / | 7 | -
-----------------
7 | / | 1 | - | 4
-----------------
- | 8 | = | 4 | ×
-----------------
4 | / | 7 | / | 9
-----------------
/ | 3 | / | 6 | B



There are 1 valid paths through the grid, producing 1 distinct equations.
7 / 8 = 7 / 3 / 6 / 4 × 9 (DRDRDLDRRUURDD)





A,,,,
└,┐,,,
,└,┐,┌,┐
,┌,┘,│,│
,└,─,┘,B


### Idea 3: A solution that uses every position
Let's find a puzzle that can be solved by visiting every square on the board (in a particular order, or course). It will almost certainly have other, shorter solutions too.

In [27]:
# Randomly generate puzzles until we get one with 
# a single solution, then show it
turns = []
while max([len(t) for t in turns], default = 0) != 24:
    grid = build_grid(5, 9, (0,0), (4,4), (2,2))
    eqns, turns = solve_AtoB(grid)

printgrid(grid)
print('\n\n')

# Report the size of the solutions set
print('There are %d valid paths through the grid, producing %d distinct equations.' 
     % (len(turns), len(set(eqns))))

# List the solutions in order of size
sort_index = sorted([(len(t), e, i) for i, (t, e) in enumerate(zip(turns,eqns))])
for _, _, i in sort_index:
    print('%s (%s)' % (eqns[i].replace('==','=').replace('*','×'), turns[i]))

# Illustrate the longest solution
print('\n\n')
start = [(i, row.index('A')) for i, row in enumerate(grid) if 'A' in row][0]
printpath(turns[sort_index[-1][-1]], start = start)

A | 7 | / | 9 | ×
-----------------
3 | × | 4 | / | 2
-----------------
- | 2 | = | 4 | +
-----------------
9 | / | 5 | × | 5
-----------------
/ | 1 | - | 4 | B



There are 16 valid paths through the grid, producing 13 distinct equations.
3 - 2 = 5 - 4 (DDRRDDRR)
3 × 2 = 4 / 4 + 5 (DRDRURDRDD)
3 - 2 = 5 / 1 - 4 (DDRRDLDRRR)
3 - 2 × 4 / 4 = 5 - 4 (DDRURRDLDDRR)
3 - 2 = 4 / 4 × 5 - 4 (DDRRURDDLDRR)
7 × 2 / 1 - 5 = 4 + 5 (RDDDDRUURRDD)
3 - 2 × 4 / 4 = 5 / 1 - 4 (DDRURRDLDLDRRR)
3 - 2 = 4 / 4 × 5 / 1 - 4 (DDRRURDDLLDRRR)
3 - 9 / 2 × 4 = 5 - 4 × 5 (DDDRUURDDDRURD)
3 - 9 / 1 / 2 × 4 = 5 - 4 × 5 (DDDDRUUURDDDRURD)
7 × 3 - 9 / 1 - 5 = 4 / 2 + 5 (RDLDDRDRUURURDDD)
7 × 3 - 9 / 1 - 5 = 4 / 2 + 5 (RDLDDRDRUUURRDDD)
7 × 3 - 9 / 1 - 5 = 4 / 2 + 5 (RDLDDDRRUURURDDD)
7 × 3 - 9 / 1 - 5 = 4 / 2 + 5 (RDLDDDRRUUURRDDD)
3 - 9 / 2 × 7 / 9 × 2 + 5 × 4 / 4 = 5 - 4 (DDDRUUURRRDDDLUULDDDRR)
3 - 9 / 1 / 2 × 7 / 9 × 2 + 5 × 4 / 4 = 5 - 4 (DDDDRUUUURRRDDDLUULDDDRR)





A,┌,─,─,┐
│,│,┌,┐,│
│,│,│,│,│
│,│,│,└,┘
└,┘,└,─,B


## "1 to 9" Puzzle #180914
Source: https://twitter.com/1to9puzzle/status/1040359407031336962
    
![The puzzle](https://pbs.twimg.com/media/DnAZXPiXsAAe3lZ.jpg)

### My solution

Let's start from scratch.

In [28]:
# Find the correct labels for all the products we might possibly encounter
from collections import defaultdict
from itertools import permutations, product

prod_dict = defaultdict(str)
max_prod = 9*8*7

# Square, cube, triangular numbers
for k in range(int(max_prod**.5)+1): prod_dict[k**2] += 'S'
for k in range(int(max_prod**.3334)+1): prod_dict[k**3] += 'C'
for k in range(1,int((max_prod*2+1)**.5)+1): prod_dict[k*(k+1)//2] += 'T'

def checkblanks(blanks, reqs):
    '''Check a given solution 
        -  `blanks` are in the order abcfihgd (start with a and move clockwise)
        -  `reqs` are given from the upper right (next to space c) clockwise'''
    b = list(blanks) # copy the values
    b.append(b[0])   # to form a cycle
    for i in range(4):
        if reqs[i] not in prod_dict[b[i*2]*b[i*2+1]*b[i*2+2]]:
            return False
    return True

try:
    import pandas as pd
    from IPython.display import display, HTML
    pandas_display = True
except:
    pandas_display = False
    
def printsoln(soln, reqs):
    '''Display a solution to this kind of puzzle'''
    grid = [[' '] * 5 for _ in range(5)]
    for k,(i,j) in enumerate([(1,1),(1,2),(1,3),(2,3),(3,3),(3,2),(3,1),(2,1)]):
        grid[i][j] = str(soln[k])
    for k,(i,j) in enumerate([(1,4),(4,3),(3,0),(0,1)]):
        grid[i][j] = str(reqs[k])
    if pandas_display:
        display(HTML(pd.DataFrame(grid).to_html(header = False, index = False)))
    else:
        print('\n'.join(' '.join(row) for row in grid))
    

In [29]:
# Get all solutions to the puzzle at hand
solns = [list(blanks) 
         for blanks in permutations(range(1,10), 8) 
         if checkblanks(blanks, ['T','T','C','S'])]

# Count them
len(solns)

75

In [30]:
# Check one of the solutions at random
printsoln(solns[17], ['T','T','C','S'])

,S,,,
,3,2,1,T
,8,,7,
C,6,9,4,
,,,T,


### Can we make a puzzle with fewer solutions?

In [31]:
# Get every possible puzzle and the number of solutions it has
puzzles = [(reqs, len([
    blanks for blanks in permutations(range(1,10), 8) 
    if checkblanks(blanks, reqs)
])) for reqs in product(['T','C','S'], repeat = 4)]

In [32]:
# Show the results
sorted(puzzles, key = lambda p: p[1])

[(('C', 'C', 'C', 'C'), 0),
 (('C', 'C', 'C', 'S'), 0),
 (('C', 'C', 'S', 'C'), 0),
 (('C', 'C', 'S', 'S'), 0),
 (('C', 'S', 'C', 'C'), 0),
 (('C', 'S', 'C', 'S'), 0),
 (('C', 'S', 'S', 'C'), 0),
 (('C', 'S', 'S', 'S'), 0),
 (('S', 'C', 'C', 'C'), 0),
 (('S', 'C', 'C', 'S'), 0),
 (('S', 'C', 'S', 'C'), 0),
 (('S', 'C', 'S', 'S'), 0),
 (('S', 'S', 'C', 'C'), 0),
 (('S', 'S', 'C', 'S'), 0),
 (('S', 'S', 'S', 'C'), 0),
 (('S', 'S', 'S', 'S'), 0),
 (('T', 'C', 'C', 'S'), 6),
 (('T', 'S', 'C', 'C'), 6),
 (('C', 'T', 'S', 'C'), 6),
 (('C', 'C', 'T', 'S'), 6),
 (('C', 'C', 'S', 'T'), 6),
 (('C', 'S', 'T', 'C'), 6),
 (('S', 'T', 'C', 'C'), 6),
 (('S', 'C', 'C', 'T'), 6),
 (('T', 'C', 'C', 'C'), 8),
 (('T', 'C', 'S', 'C'), 8),
 (('C', 'T', 'C', 'C'), 8),
 (('C', 'T', 'C', 'S'), 8),
 (('C', 'C', 'T', 'C'), 8),
 (('C', 'C', 'C', 'T'), 8),
 (('C', 'S', 'C', 'T'), 8),
 (('S', 'C', 'T', 'C'), 8),
 (('T', 'C', 'T', 'S'), 10),
 (('T', 'C', 'S', 'S'), 10),
 (('T', 'S', 'T', 'C'), 10),
 (('T', 'S', 'S',

#### There's a lot of redundancy in that list...

In [33]:
# Let's create a function to map every sequence of letters to
# a "canonical" form to eliminate the equivalent sequences.
# e.g. SCTS is equivalent to CSST, but CSTS is not.
def canonical(tup):
    '''Map all equivalent sequences to a single "canonical" form.'''
    variations = [tup[k:] + tup[:k] for k in range(len(tup))]
    variations += [tup[::-1][k:] + tup[::-1][:k] for k in range(len(tup))]
    return sorted(variations)[0]

# Couple of tests
assert canonical(('S','C','T','S')) == canonical(('C','S','S','T'))
assert canonical(('C','S','T','S')) != canonical(('C','S','S','T'))

In [34]:
# Get every possible puzzle, minus redundancies, along with the solution count
puzzles = [(reqs, len([
    blanks for blanks in permutations(range(1,10), 8) 
    if checkblanks(blanks, reqs)
])) for reqs in set(canonical(p) for p in product(['T','C','S'], repeat = 4))]

In [35]:
# Report the results!
for p, s in sorted(puzzles, key = lambda p: p[1]):
    print('Puzzle', ''.join(p), 'has', s, 'solutions')

Puzzle SSSS has 0 solutions
Puzzle CCCS has 0 solutions
Puzzle CCSS has 0 solutions
Puzzle CSSS has 0 solutions
Puzzle CSCS has 0 solutions
Puzzle CCCC has 0 solutions
Puzzle CCST has 6 solutions
Puzzle CCCT has 8 solutions
Puzzle CSCT has 8 solutions
Puzzle CTST has 10 solutions
Puzzle CSST has 10 solutions
Puzzle CTCT has 12 solutions
Puzzle CSTS has 16 solutions
Puzzle STST has 28 solutions
Puzzle SSST has 32 solutions
Puzzle CCTT has 52 solutions
Puzzle CSTT has 75 solutions
Puzzle CTTT has 78 solutions
Puzzle TTTT has 128 solutions
Puzzle SSTT has 134 solutions
Puzzle STTT has 140 solutions


In [36]:
# Show all the solutions of the CCST puzzle type (the least solvable type)
printsoln(['?']*8, ['C','C','S','T'])
print('\n')
solns = [list(blanks) 
         for blanks in permutations(range(1,10), 8) 
         if checkblanks(blanks, ['C','C','S','T'])]

for s in solns:
    printsoln(s, ['C','C','S','T'])
    print('\n')

,T,,,
,?,?,?,C
,?,,?,
S,?,?,?,
,,,C,


,T,,,
,1,2,4,C
,5,,9,
S,3,8,6,
,,,C,


,T,,,
,1,2,4,C
,7,,9,
S,3,8,6,
,,,C,


,T,,,
,1,3,9,C
,5,,6,
S,2,8,4,
,,,C,


,T,,,
,3,1,9,C
,5,,6,
S,8,2,4,
,,,C,


,T,,,
,4,2,1,C
,5,,9,
S,6,8,3,
,,,C,


,T,,,
,4,6,9,C
,7,,3,
S,1,2,8,
,,,C,


### Can we make a puzzle where the given products increase incrementally?

In [37]:
def diffs(li):
    return [b - a for a,b in zip(li[:-1],li[1:])]

def prods(blanks):
    b = list(blanks)
    b.append(b[0])
    return tuple(b[i*2]*b[i*2+1]*b[i*2+2] for i in range(4))

for puz in set(
    canonical(prods(b)) 
    for b in permutations(range(1,10), 8) 
    if len(set(diffs(prods(b)))) == 1
):
    printsoln(['?']*8, puz)
    print('\n')

,120,,,
,?,?,?,12
,?,,?,
84,?,?,?,
,,,48,


,60,,,
,?,?,?,24
,?,,?,
48,?,?,?,
,,,36,


## "1 to 9" puzzle #1837
Source: https://twitter.com/1to9puzzle/status/1040589799260909568

![The puzzle](https://pbs.twimg.com/media/DnDqY4bW0AEXDHs.jpg)

### My solution
Let's start from scratch.

In [38]:
from copy import deepcopy
from itertools import product

In [39]:
def checkrow(row):
    '''Make sure a given row (or column/block) isn't breaking the rules.'''
    row_clean = [r for r in row if r > 0]
    return len(set(row_clean)) == len(row_clean)

def cols(grid):
    '''Get all the columns of a grid (essentially, transpose the grid).'''
    return [[e[i] for e in grid] for i in range(len(grid[0]))]

def col(grid, k):
    '''Get a single column of a grid.'''
    return [e[k] for e in grid]

def blocks(grid):
    '''Get all the 3x3 blocks of a grid.'''
    return [
        grid[i][j:j+3] + grid[i+1][j:j+3] + grid[i+2][j:j+3]
        for i,j in product(range(0,9,3), repeat = 2)
    ]

def block(grid, i, j):
    '''Get a single block of a grid.'''
    return (
        grid[i//3*3][j//3*3:j//3*3+3] + 
        grid[i//3*3+1][j//3*3:j//3*3+3] + 
        grid[i//3*3+2][j//3*3:j//3*3+3]
    )

primes = {11, 13, 17, 19, 23, 29, 31, 37, 
          41, 43, 47, 53, 59, 61, 67, 71, 
          73, 79, 83, 89, 97}

def labels(grid):
    '''Find the labels (i.e. the colors) for the entire grid.'''
    labs = [[0]*9 for _ in range(9)]
    for i, row in enumerate(grid):
        for j in range(8):
            if row[j]*10 + row[j+1] in primes:
                labs[i][j] += 1
                labs[i][j+1] += 1
    for i, row in enumerate(cols(grid)):
        for j in range(8):
            if row[j]*10 + row[j+1] in primes:
                labs[j][i] += 1
                labs[j+1][i] += 1
    return labs

def label(grid, i, j):
    '''Find the label for one spot in the grid.'''    
    lab = 0
    if i < 8 and grid[i][j] * 10 + grid[i+1][j] in primes: lab += 1
    if j < 8 and grid[i][j] * 10 + grid[i][j+1] in primes: lab += 1
    if i > 0 and grid[i-1][j] * 10 + grid[i][j] in primes: lab += 1
    if j > 0 and grid[i][j-1] * 10 + grid[i][j] in primes: lab += 1
    return lab
        
def checkgrid(grid, grid_reqs):
    '''Check that no rules are violated in an entire grid.'''
    if not all(checkrow(g) for g in grid):
        return False
    if not all(checkrow(g) for g in cols(grid)):
        return False
    if not all(checkrow(g) for g in blocks(grid)):
        return False
    if not all(all(eg == el for eg, el in zip(g, l))
               for g,l in zip(grid_reqs, labels(grid))):
        return False
    return True

def printgrid(grid):
    '''Neatly print out a grid.'''
    print(('\n' + '-' * (4*len(grid[0]) - 3) + '\n')
          .join([' | '.join(str(i) for i in g) for g in grid]))  
    
def solve_sudoku(givens, grid_reqs):
    '''Find all solutions that follow the rules of sudoku
       and the given prime color labels'''

    solutions = []
    
    # Size of the problem (hard-coded!)
    n = 9

    # The initial grid
    grid = [[0] * n for _ in range(n)]
    
    # Fill in the given spaces and mark them as never to be altered
    for (i,j), val in givens:
        grid[i][j] = val
    skips = [(i,j) for (i,j),val in givens]
    
    # Starting position in the grid (upper left)
    currx, curry = 0, 0

    # Keep working until all possibilites are exhausted
    # (so if there are multiple solutions, we can find all of them)
    while curry >= 0:

        # At the current position in the grid, try the next digit
        grid[curry][currx] += 1

        # If there are no more digits to try, back up to the previous position
        if grid[curry][currx] > 9:
            grid[curry][currx] = 0
            backup = True
            while backup:
                currx -= 1
                if currx < 0:
                    currx  = n-1
                    curry -= 1
                backup = ((curry, currx) in skips)
            continue
        
        # If we have found a solution, save it
        if (currx, curry) == (n-1,n-1):
            if checkgrid(grid, grid_reqs): 
                solutions.append(deepcopy(grid))                
            
        # Otherwise, if we're not currently breaking 
        # any obvious rules, proceed to the next position
        elif (checkrow(grid[curry]) and 
              checkrow(col(grid, currx)) and
              checkrow(block(grid, curry, currx)) and
              (curry == 0 or label(grid, curry-1, currx) 
                          == grid_reqs[curry-1][currx])):
            proceed = True
            while proceed:
                currx += 1
                if currx > n-1:
                    currx  = 0
                    curry += 1
                proceed = ((curry, currx) in skips)
                
    return solutions


In [40]:
# Solve the given puzzle
soln = solve_sudoku(
    
    givens = [((1,1), 8), ((1,4), 6), ((1,7), 3),
              ((4,1), 9), ((4,4), 1), ((4,7), 7),
              ((7,1), 1), ((7,4), 7), ((7,7), 5)],
    
    grid_reqs = [[1,2,2,1,0,1,1,1,1],
                 [0,1,2,1,1,2,1,2,0],
                 [2,1,0,1,1,1,0,1,0],
                 [1,1,2,2,1,0,0,0,1],
                 [1,0,0,1,0,0,2,3,2],
                 [2,0,1,2,0,1,4,2,0],
                 [1,1,1,1,0,2,2,0,1],
                 [1,1,1,1,2,3,1,1,2],
                 [1,0,1,0,0,2,2,0,0]]
)

# Show only the solution(s) with "b > a"
for s in soln: 
    if s[0][5] > s[0][4]: 
        printgrid(s)
        print('\n\n')

print('(%d solutions in total)' % len(soln))

5 | 3 | 1 | 4 | 2 | 8 | 9 | 6 | 7
---------------------------------
2 | 8 | 9 | 1 | 6 | 7 | 5 | 3 | 4
---------------------------------
4 | 7 | 6 | 5 | 3 | 9 | 8 | 1 | 2
---------------------------------
1 | 4 | 3 | 7 | 9 | 5 | 6 | 2 | 8
---------------------------------
6 | 9 | 5 | 8 | 1 | 2 | 4 | 7 | 3
---------------------------------
7 | 2 | 8 | 3 | 4 | 6 | 1 | 9 | 5
---------------------------------
3 | 6 | 2 | 9 | 5 | 4 | 7 | 8 | 1
---------------------------------
8 | 1 | 4 | 6 | 7 | 3 | 2 | 5 | 9
---------------------------------
9 | 5 | 7 | 2 | 8 | 1 | 3 | 4 | 6



(4 solutions in total)


## "1 to 9" puzzle #1841
Source: https://twitter.com/1to9puzzle/status/1050738145216647169

![The puzzle](https://pbs.twimg.com/media/DpT2qe2WsAA_n3k.jpg)

### My solution
Just need to slightly alter the previous one.

In [41]:
mults = set(3*i for i in range(4,34))

def labels(grid):
    '''Find the labels (i.e. the colors) for the entire grid.'''
    labs = [[0]*9 for _ in range(9)]
    for i, row in enumerate(grid):
        for j in range(8):
            if row[j]*10 + row[j+1] in mults:
                labs[i][j] += 1
                labs[i][j+1] += 1
    for i, row in enumerate(cols(grid)):
        for j in range(8):
            if row[j]*10 + row[j+1] in mults:
                labs[j][i] += 1
                labs[j+1][i] += 1
    return labs

def label(grid, i, j):
    '''Find the label for one spot in the grid.'''    
    lab = 0
    if i < 8 and grid[i][j] * 10 + grid[i+1][j] in mults: lab += 1
    if j < 8 and grid[i][j] * 10 + grid[i][j+1] in mults: lab += 1
    if i > 0 and grid[i-1][j] * 10 + grid[i][j] in mults: lab += 1
    if j > 0 and grid[i][j-1] * 10 + grid[i][j] in mults: lab += 1
    return lab

In [42]:
# Solve the given puzzle
soln = solve_sudoku(
    
    givens = [((1,1), 2), ((0,5), 1), ((2,7), 3),
              ((3,6), 5), ((4,4), 4), ((5,0), 4),
              ((7,2), 7), ((7,8), 8), ((8,3), 6)],
    
    grid_reqs = [[0,1,1,0,0,1,2,1,0],
                 [1,2,1,2,1,0,1,1,1],
                 [1,3,1,1,0,2,2,1,3],
                 [0,1,0,1,0,2,3,2,1],
                 [0,0,0,1,1,0,0,2,1],
                 [1,0,0,1,1,0,2,1,1],
                 [1,1,0,3,1,1,2,1,2],
                 [0,1,1,2,0,1,0,0,1],
                 [0,1,1,0,1,2,0,1,2]]
)

# Show only the solution(s) with "a > b and c > d"
for s in soln: 
    if s[0][2] > s[1][2] and s[2][0] > s[2][1]: 
        printgrid(s)
        print('\n\n')

print('(%d solutions in total)' % len(soln))

3 | 4 | 9 | 7 | 6 | 1 | 8 | 5 | 2
---------------------------------
5 | 2 | 6 | 4 | 8 | 3 | 7 | 1 | 9
---------------------------------
7 | 1 | 8 | 2 | 9 | 5 | 4 | 3 | 6
---------------------------------
6 | 8 | 2 | 9 | 1 | 7 | 5 | 4 | 3
---------------------------------
1 | 9 | 5 | 3 | 4 | 6 | 2 | 8 | 7
---------------------------------
4 | 7 | 3 | 8 | 5 | 2 | 9 | 6 | 1
---------------------------------
8 | 3 | 4 | 1 | 2 | 9 | 6 | 7 | 5
---------------------------------
2 | 6 | 7 | 5 | 3 | 4 | 1 | 9 | 8
---------------------------------
9 | 5 | 1 | 6 | 7 | 8 | 3 | 2 | 4



(6 solutions in total)
